In [34]:
def setBackgroundVariables_I(Nx,Ny,e_r,e_0,mu,dt,dx,dy):
    #----------------------------------------
    #-          PML boundaries              -
    #----------------------------------------
    dLayers = 10                                 #number of layers
    order   = 3                                              #order of polynomial of conductivity values
    s_x = np.zeros((Ny,Nx))
    s_y = np.zeros((Ny,Nx))       #electric conductivities

    for iX in range(Nx):
        s_x[iX, :dLayers] = np.power(list(range(dLayers-1,-1,-1)), order) / (dLayers-1)**order
        s_x[iX,-dLayers:] = np.power(list(range(dLayers)), order) / (dLayers-1)**order

    for iY in range(Ny):
        s_y[:dLayers,iY]  = np.power(list(range(dLayers-1,-1,-1)), order) / (dLayers-1)**order
        s_y[-dLayers:,iY] = np.power(list(range(dLayers)), order) / (dLayers-1)**order

    ss_x = s_x * mu / e_0
    ss_y = s_y * mu / e_0                       #magnetic "conductivities"
    #----------------------------------------
    #-         wall material props.         -
    #----------------------------------------
    wall_permittivity = 3.0
    door_permittivity = 2.0

    e_r[dLayers-1:-dLayers,dLayers-1:-dLayers] = makeRoom(e_r[dLayers-1:-dLayers,dLayers-1:-dLayers], wall_permittivity, door_permittivity)
    
    e_r_temp = e_r-1
    ind = np.nonzero(e_r)
    for el in ind:
        s_x[el[0], el[1]] = e_r[el[0], el[1]]/40
        s_y[el[0], el[1]] = e_r[el[0], el[1]]/40
    

    Cezx1 = np.divide((e_0*e_r-0.5*dt*s_x),(e_0*e_r+0.5*dt*s_x))     #Ezx first term coefficient
    Cezx2 = dt/(dx*(e_0*e_r+0.5*dt*s_x))                             #Ezx second term coefficient
    Cezy1 = np.divide((e_0*e_r-0.5*dt*s_y),(e_0*e_r+0.5*dt*s_y))     #Ezy first term coefficient
    Cezy2 = dt/(dy*(e_0*e_r+0.5*dt*s_y))                             #Ezy second term coefficient

    Chx1  = np.divide((mu-0.5*dt*ss_x),(mu+0.5*dt*ss_x))             #Hx first term coefficient
    Chx2  = dt/(dy*(mu+0.5*dt*ss_x))                                 #Hx second term coefficient

    Chy1  = np.divide((mu-0.5*dt*ss_y),(mu+0.5*dt*ss_y))             #Hy second term coefficient
    Chy2  = dt/(dx*(mu+0.5*dt*ss_y))                                 #Hy second term coefficient
    return [Cezx1,Cezx2,Cezy1,Cezy2,Chx1,Chx2,Chy1,Chy2,e_r]